In [1]:
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers
from gutenberg.query import get_metadata
import flair

ModuleNotFoundError: No module named 'flair'

In [2]:
text = strip_headers(load_etext(2701)).strip()
with open("mobydick.txt","w+") as f:
    f.write(text)
print("done")

done


In [3]:
import json
data = json.load(open("gutenberg-response.json"))

In [4]:
import pandas as pd
def extract(d):
    r = {}
    r['languages'] = d['languages']
    r['id'] = d['id']
    r['title'] = d['title']
    births = [a.get('birth_year') for a in d['authors']]
    death = [a.get('death_year') for a in d['authors']]
    min_birth = min(filter(bool, births), default=99999)
    max_death = max(filter(bool, death), default=-1)
    
    r['year_range'] = [min_birth, max_death]
    return r
            
res  =[extract(x) for x in data['results']]
df = pd.DataFrame(data['results'])

In [5]:
from collections import Counter
print("from\n", Counter(x['year_range'][0] for x in res))
print("to", Counter(x['year_range'][1] for x in res))

from
 Counter({1812: 3, 1775: 2, 1828: 2, 99999: 2, 1854: 2, 1835: 2, 1797: 1, 1667: 1, 1857: 1, 1847: 1, 1783: 1, 1819: 1, 1832: 1, 1850: 1, 1883: 1, -750: 1, 1785: 1, 1860: 1, 1859: 1, 1588: 1, -428: 1, 1816: 1, 1469: 1, 1868: 1, 1818: 1})
to Counter({1870: 3, 1910: 3, 1817: 2, 1924: 2, -1: 2, 1900: 2, 1851: 1, 1745: 1, 1912: 1, 1906: 1, 1859: 1, 1891: 1, 1898: 1, 1894: 1, -650: 1, 1863: 1, 1935: 1, 1930: 1, 1679: 1, -348: 1, 1855: 1, 1527: 1, 1963: 1, 1895: 1})


,authors,bookshelves,copyright,download_count,formats,id,languages,media_type,subjects,title
0,"[{'name': 'Austen, Jane', 'birth_year': 1775, ...","[Best Books Ever Listings, Harvard Classics]",False,40842,{'application/rdf+xml': 'http://www.gutenberg....,1342,[en],Text,"[Courtship -- Fiction, Domestic fiction, Engla...",Pride and Prejudice
1,"[{'name': 'Shelley, Mary Wollstonecraft', 'bir...","[Gothic Fiction, Movie Books, Precursors of Sc...",False,31848,{'application/x-mobipocket-ebook': 'http://www...,84,[en],Text,[Frankenstein's monster (Fictitious character)...,"Frankenstein; Or, The Modern Prometheus"
2,"[{'name': 'Swift, Jonathan', 'birth_year': 166...",[],False,20875,{'text/plain': 'http://www.gutenberg.org/ebook...,1080,[en],Text,[Ireland -- Politics and government -- 18th ce...,A Modest Proposal: For preventing the children...
3,"[{'name': 'Conrad, Joseph', 'birth_year': 1857...","[Best Books Ever Listings, Movie Books]",False,20554,{'application/rdf+xml': 'http://www.gutenberg....,219,[en],Text,"[Africa -- Fiction, Degeneration -- Fiction, E...",Heart of Darkness
4,"[{'name': 'Stoker, Bram', 'birth_year': 1847, ...","[Gothic Fiction, Horror, Movie Books, Mystery ...",False,19291,{'text/plain': 'http://www.gutenberg.org/ebook...,345,[en],Text,"[Dracula, Count (Fictitious character) -- Fict...",Dracula
5,"[{'name': 'Ibsen, Henrik', 'birth_year': 1828,...",[Best Books Ever Listings],False,17200,{'text/plain': 'http://www.gutenberg.org/ebook...,2542,[en],Text,"[Man-woman relationships -- Drama, Marriage --...",A Doll's House : a play
6,"[{'name': 'Irving, Washington', 'birth_year': ...","[Children's Literature, Harvard Classics]",False,16859,{'application/epub+zip': 'http://www.gutenberg...,41,[en],Text,"[Ghosts -- Fiction, New York (State) -- Histor...",The Legend of Sleepy Hollow
7,"[{'name': 'Dickens, Charles', 'birth_year': 18...",[Historical Fiction],False,16315,{'application/rdf+xml': 'http://www.gutenberg....,98,[en],Text,"[British -- France -- Paris -- Fiction, Execut...",A Tale of Two Cities
8,[],[Poetry],False,15734,{'text/plain': 'http://www.gutenberg.org/ebook...,16328,[en],Text,"[Dragons -- Poetry, Epic poetry, English (Old)...",Beowulf: An Anglo-Saxon Epic Poem
9,"[{'name': 'Melville, Herman', 'birth_year': 18...",[Best Books Ever Listings],False,15401,{'application/x-mobipocket-ebook': 'http://www...,2701,[en],Text,"[Adventure stories, Ahab, Captain (Fictitious ...","Moby Dick; Or, The Whale"
